Make initializations

In [1]:
import cbh_data_definitions
import pathlib
import os

import optuna
import pytorch_lightning as pl
import mlflow
from ray import tune
import ray
import ray.tune
import ray.tune.search
import ray.tune.search.optuna
from ray.tune.search.optuna import OptunaSearch
from ray.tune.integration.mlflow import mlflow_mixin
from ray.tune.search import ConcurrencyLimiter
import numpy as np
import datetime
import cbh_torch_MLP

In [2]:
# intialize some settings: mlflow, data directory, resources
root_data_directory = pathlib.Path(os.environ["SCRATCH"]) / "cbh_data"

dev_data_path = root_data_directory / "analysis_ready" / "dev_randomized.zarr"
training_data_path = root_data_directory / "analysis_ready" / "train_randomized.zarr"

mlflow_command_line_run = """
    mlflow server --port 5001 --backend-store-uri sqlite:///mlflowSQLserver.db  --default-artifact-root ./mlflow_artifacts/
"""
mlflow_server_address = 'vld425'
mlflow_server_port = 5001
mlflow_server_uri = f'http://{mlflow_server_address}:{mlflow_server_port:d}'
mlflow_artifact_root = pathlib.Path('./mlflow_artifacts/')

hparams_for_mlflow = {}

redefine data

In [3]:
# init data
(
    train_input,
    train_labels,
    _,
) = cbh_data_definitions.load_data_from_zarr(training_data_path)

(
    dev_input, 
    dev_labels, 
    _
) = cbh_data_definitions.load_data_from_zarr(dev_data_path)

# the cloud volume is not needed for the task, so isn't saved on the load
# show a chunk
train_input

Loaded zarr, file information:
 Name              : /
Type              : zarr.hierarchy.Group
Read-only         : False
Synchronizer type : zarr.sync.ThreadSynchronizer
Store type        : zarr.storage.DirectoryStore
No. members       : 2
No. arrays        : 2
No. groups        : 0
Arrays            : cloud_base_label_y.zarr, humidity_temp_pressure_x.zarr
 

Loaded zarr, file information:
 Name              : /
Type              : zarr.hierarchy.Group
Read-only         : False
Synchronizer type : zarr.sync.ThreadSynchronizer
Store type        : zarr.storage.DirectoryStore
No. members       : 2
No. arrays        : 2
No. groups        : 0
Arrays            : cloud_base_label_y.zarr, humidity_temp_pressure_x.zarr
 



dask.array<from-zarr, shape=(111820800, 70, 3), dtype=float32, chunksize=(2329600, 70, 3), chunktype=numpy.ndarray>

In [4]:
# limit the data by a factor for less data in a tuning trial
factors_of_chunk = [n for n in range(1, train_input.chunksize[0] + 1) if train_input.chunksize[0] % n == 0]
print("Factors of chunk: ", factors_of_chunk)
hparams_for_mlflow['Limited sample number'] =  -1

Factors of chunk:  [1, 2, 4, 5, 7, 8, 10, 13, 14, 16, 20, 25, 26, 28, 32, 35, 40, 50, 52, 56, 64, 65, 70, 80, 91, 100, 104, 112, 128, 130, 140, 160, 175, 182, 200, 208, 224, 256, 260, 280, 320, 325, 350, 364, 400, 416, 448, 455, 512, 520, 560, 640, 650, 700, 728, 800, 832, 896, 910, 1024, 1040, 1120, 1280, 1300, 1400, 1456, 1600, 1664, 1792, 1820, 2080, 2240, 2275, 2560, 2600, 2800, 2912, 3200, 3328, 3584, 3640, 4160, 4480, 4550, 5120, 5200, 5600, 5824, 6400, 6656, 7168, 7280, 8320, 8960, 9100, 10400, 11200, 11648, 12800, 13312, 14560, 16640, 17920, 18200, 20800, 22400, 23296, 25600, 29120, 33280, 35840, 36400, 41600, 44800, 46592, 58240, 66560, 72800, 83200, 89600, 93184, 116480, 145600, 166400, 179200, 232960, 291200, 332800, 465920, 582400, 1164800, 2329600]


setup study

In [5]:
# DEFINE ALL SETTINGS FOR TRAINING, includes hparam space
experiment_name = 'cbh-hparam-tuning'
CPU_COUNT = 4
RAM_GB = 6
hparams_for_mlflow['CPU Count'] = CPU_COUNT
hparams_for_mlflow['Compute Memory'] = RAM_GB
thread_count_for_dask = CPU_COUNT
dataset_method = '1chunk'
randomize_chunkwise_1chunk = False
shuffle_train_data = False
collate_fn = None # alt: cbh_data_definitions.dataloader_collate_with_dask
num_workers_dataloader = 0 # alt: CPU_COUNT +-
global_trail_number = 0
max_time_for_trial = "00:02:00:00"  # dd:hh:mm:ss
hparams_for_mlflow["Training timeout"] = max_time_for_trial

max_node_num_exclusive = 513
factors_for_hparam_choice = [factor for factor in factors_of_chunk if (factor<3300 and factor>3)]
mlp_search_space = {
    "epoch": 1,
    "lr": tune.quniform(0.0001, 0.01, 0.00005),
    "data_limit": tune.randint(4, int(len(train_labels.chunks[0]) / 4)), # multiple chunk ind by chunklen: train_input.chunksize[0])
    # "activation": tune.choice(["relu", "tanh"]),
    "batch_size": tune.choice(factors_for_hparam_choice),
    "arch_name":"MLP",
    "hidden_layers":tune.randint(1,11),
    "activation":tune.choice(["relu", "tanh"]),
    "input_size":(train_input.shape[2] * train_input.shape[1]),
    "output_size": train_input.shape[1],
    "layer_node_num": tune.sample_from(lambda spec: 8*np.random.randint(1,int(max_node_num_exclusive/8), size=spec.config.hidden_layers)),
}

In [6]:
class MLFlowLogger(pl.loggers.MLFlowLogger): #overwrite mlflogger
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    def after_save_checkpoint(self, model_checkpoint: pl.callbacks.ModelCheckpoint) -> None:
        """
        Called after model checkpoint callback saves a new checkpoint.
        """
        best_chkpt = torch.load(model_checkpoint.best_model_path)
        checkpoint_for_mlflow = {
            "val loss": float(best_chkpt['callbacks'][list(key for key in list(best_chkpt['callbacks'].keys()) if "ModelCheckpoint" in key)[0]]['current_score']),
            "train loss at step-1": list(train_loss_metric.value for train_loss_metric in mlf_logger._mlflow_client.get_metric_history(run.info.run_id, "Train loss") if (int(train_loss_metric.step) == int(best_chkpt['global_step']-1)))[0],
            "global_step": best_chkpt['global_step'],
            "model_state_dict": best_chkpt['state_dict'],
            "checkpoint": best_chkpt,
        }
        with TemporaryDirectory() as tmpdirname:
            f_name = os.path.join(tmpdirname, f"{run.info.run_id}-best_model_checkpoint-step_{best_chkpt['global_step']}.pt")
            torch.save(checkpoint_for_mlflow, f_name)
            mlflow.log_artifact(f_name)

mlflow.set_tracking_uri(mlflow_server_uri)
# make vars global
mlf_exp = None
mlf_exp_id = None
try: 
    print('Creating experiment')
    mlf_exp_id = mlflow.create_experiment(experiment_name)
    mlf_exp = mlflow.get_experiment(mlf_exp_id)
except mlflow.exceptions.RestException as e:
    print("Caught")
    print(e)
    mlf_exp = mlflow.get_experiment_by_name(experiment_name)
print("Success")

mlp_search_space["mlflow"] = {
    "tracking_uri":mlflow_server_uri,
    "experiment_id":mlf_exp_id,
    "experiment_name":experiment_name,
}

Creating experiment
Caught
RESOURCE_ALREADY_EXISTS: Experiment(name=cbh-hparam-tuning) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('cbh-hparam-tuning', '', 'active', 1668101925780, 1668101925780)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)
Success


In [9]:
@mlflow_mixin
def objective(ray_config):
    # def model hparams with config
    print(ray_config)
    print(dir(ray_config))
    # def data
    datamodule = cbh_data_definitions.CBH_DataModule(
        train_input, train_labels,
        dev_input, dev_labels,
        thread_count_for_dask,
        ray_config['batch_size'],
        num_workers = num_workers_dataloader,
        collate_fn = collate_fn,
        shuffle = shuffle_train_data,
        randomize_chunkwise = randomize_chunkwise_1chunk,
        method=dataset_method,
    )
    #def model
    model = cbh_torch_MLP.CloudBaseMLP(
        ray_config['input_size'],
        ray_config['layer_node_num'],
        ray_config['output_size'],
        ray_config['hidden_layers'],
        ray_config['activation'],
        ray_config['lr'],
    )
    # def experiment naming
    timestamp_template = '{dt.year:04d}{dt.month:02d}{dt.day:02d}T{dt.hour:02d}{dt.minute:02d}{dt.second:02d}'
    run_name_template = 'cbh_challenge_{network_name}_' + timestamp_template
    global global_trail_number
    current_run_name = run_name_template.format(network_name=model.__class__.__name__,
                                                    dt=datetime.datetime.now()
                                                   )
    
    # begin mlflow experiment run
    with mlflow.start_run(experiment_id=mlf_exp.experiment_id, run_name=current_run_name, nested=True) as run:
    
        mlflow.pytorch.autolog()
        mlf_logger = MLFlowLogger(experiment_name=experiment_name, tracking_uri=mlflow_server_uri, run_id=run.info.run_id)
        return {"val_loss":1.0}
        # define trainer
        time_for_checkpoint = datetime.timedelta(minutes=15)
        checkpoint_callback = pl.callbacks.ModelCheckpoint(
            train_time_interval=time_for_checkpoint,
            dirpath=run.info.artifact_uri,
            monitor="Val loss",
            save_on_train_epoch_end=False,
            mode="min"
        )
        callbacks = [checkpoint_callback, RichProgressBar()]
        
        trainer_hparams = {
            'max_epochs':epochs,
            'deterministic':True,
            'val_check_interval':0.05, # val every percentage of the data
            'devices':"auto",
            'accelerator':"auto",
            'max_time':max_time,
            'replace_sampler_ddp':False,
            'enable_checkpointing':True,
            'strategy':None,
            'callbacks':callbacks,
            'logger':mlf_logger,
        }
        
        hparams_for_mlflow["Trainer hparams"] = trainer_hparams
        mlf_logger.log_hyperparams(hparams_for_mlflow)
        
        trainer = pl.Trainer(
            **trainer_hparams
        )

        trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)
        path_to_save = '{dt.year:04d}{dt.month:02d}{dt.day:02d}-{dt.hour:02d}{dt.minute:02d}{dt.second:02d}'.format(dt=datetime.datetime.now())
        trainer.save_checkpoint(filepath=run.info.artifact_uri + f'/post_epoch_modelchkpt_{path_to_save}')
    return trainer.callback_metrics["val_acc"].item()

In [10]:
searcher = OptunaSearch(metric=["val_loss"], mode=["min"])
algo = ConcurrencyLimiter(searcher, max_concurrent=int(CPU_COUNT*(3/4)))
num_hparam_trials = 4

tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        # search_alg=algo,
        num_samples=num_hparam_trials,
    ),
    param_space=mlp_search_space,
)
results = tuner.fit()

Trial name,status,loc,activation,batch_size,data_limit,hidden_layers,layer_node_num,lr,iter,total time (s),val_loss
objective_ab147_00000,TERMINATED,10.152.49.117:68201,tanh,5,7,5,[256 24 304 32_efd0,0.0073,1,0.0885732,1
objective_ab147_00001,TERMINATED,10.152.49.117:68287,tanh,64,8,10,[ 64 192 280 38_c0f0,0.00645,1,0.126102,1
objective_ab147_00002,TERMINATED,10.152.49.117:68333,relu,140,4,2,[152 176],0.00155,1,0.103521,1
objective_ab147_00003,TERMINATED,10.152.49.117:68369,relu,16,4,9,[440 8 312 40_9e30,0.00865,1,0.245827,1


(objective pid=68201) {'epoch': 1, 'lr': 0.0073, 'data_limit': 7, 'batch_size': 5, 'arch_name': 'MLP', 'hidden_layers': 5, 'activation': 'tanh', 'input_size': 210, 'output_size': 70, 'layer_node_num': array([256,  24, 304, 328, 192]), 'mlflow': {'tracking_uri': 'http://vld425:5001', 'experiment_id': None, 'experiment_name': 'cbh-hparam-tuning'}}
(objective pid=68201) ['__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']
(objective pid=68287) {'epoch': 1, 'lr': 0.00645, 'data_limit': 8, '

2022-11-10 17:40:14,904	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'layer_node_num': array([256,  24, 304, 328, 192])}
2022-11-10 17:40:14,950	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'layer_node_num': array([152, 176])}
2022-11-10 17:40:14,977	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'layer_node_num': array([ 64, 192, 280, 384,  56, 416, 216, 464, 176, 504])}


Result for objective_ab147_00000:
  date: 2022-11-10_17-39-55
  done: true
  experiment_id: 216b2145504b40b0acfeb2db88751245
  experiment_tag: 0_activation=tanh,batch_size=5,data_limit=7,hidden_layers=5,layer_node_num=256_24_304_328_192,lr=0.0073
  hostname: vld425
  iterations_since_restore: 1
  node_ip: 10.152.49.117
  pid: 68201
  time_since_restore: 0.08857321739196777
  time_this_iter_s: 0.08857321739196777
  time_total_s: 0.08857321739196777
  timestamp: 1668101995
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: ab147_00000
  val_loss: 1.0
  warmup_time: 0.26276159286499023
  
Result for objective_ab147_00002:
  date: 2022-11-10_17-40-09
  done: false
  experiment_id: 8e857ff3ce074279adf98c569d083f6f
  hostname: vld425
  iterations_since_restore: 1
  node_ip: 10.152.49.117
  pid: 68333
  time_since_restore: 0.10352063179016113
  time_this_iter_s: 0.10352063179016113
  time_total_s: 0.10352063179016113
  timestamp: 1668102009
  timesteps_since_restore: 0
  trainin

2022-11-10 17:40:15,138	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'layer_node_num': array([440,   8, 312, 400, 128, 312, 208,  80,  48])}


Result for objective_ab147_00003:
  date: 2022-11-10_17-40-15
  done: false
  experiment_id: a5f8f16baa4e442186e2ec5d16f7bbcd
  hostname: vld425
  iterations_since_restore: 1
  node_ip: 10.152.49.117
  pid: 68369
  time_since_restore: 0.24582695960998535
  time_this_iter_s: 0.24582695960998535
  time_total_s: 0.24582695960998535
  timestamp: 1668102015
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: ab147_00003
  val_loss: 1.0
  warmup_time: 0.22005677223205566
  
Result for objective_ab147_00003:
  date: 2022-11-10_17-40-15
  done: true
  experiment_id: a5f8f16baa4e442186e2ec5d16f7bbcd
  experiment_tag: 3_activation=relu,batch_size=16,data_limit=4,hidden_layers=9,layer_node_num=440_8_312_400_128_312_208_80_48,lr=0.0086
  hostname: vld425
  iterations_since_restore: 1
  node_ip: 10.152.49.117
  pid: 68369
  time_since_restore: 0.24582695960998535
  time_this_iter_s: 0.24582695960998535
  time_total_s: 0.24582695960998535
  timestamp: 1668102015
  timesteps_since_restor

2022-11-10 17:40:15,331	INFO tune.py:758 -- Total run time: 28.79 seconds (28.59 seconds for the tuning loop).


ensure mlflow

run study

eval